# Data Analysis & Data Scientist Canadian Salaries

In [1]:
import pandas as pd, re 
pd.set_option('display.max_rows',900)

## Collecting Data Sources & formatting them


### Starting with Statcan 2021 census information
https://www.jobbank.gc.ca/marketreport/wages-occupation/17882/ca

In [107]:
file = r'C:\Users\Andre\OneDrive\Desktop\Data analysis\Salary_Analysis\StatCan_salary.csv'
dfWagesOG = pd.read_csv(file,low_memory=False)


In [108]:
dfWagesOG.columns

Index(['NOC_CNP', 'NOC_TITLE_ENG', 'NOC_TITLE_FRA', 'prov', 'ER_Code_Code_RE',
       'ER_Name', 'Nom_RE', 'Low_Wage_Salaire_Minium',
       'Median_Wage_Salaire_Median', 'High_Wage_Salaire_Maximal',
       'Average_Wage_Salaire_Moyen', 'Data_Source_E', 'Data_Source_F',
       'Reference_Period', 'Revision_Date_Date_revision',
       'Annual_Wage_Flag_Salaire_annuel', 'Wage_Comment_E', 'Wage_Comment_F'],
      dtype='object')

In [109]:
df=dfWagesOG.copy()
dfDS = df[df['NOC_TITLE_ENG'].str.contains('Data sci',na=False) ].copy()
dfDS.describe()

,Low_Wage_Salaire_Minium,Median_Wage_Salaire_Median,High_Wage_Salaire_Maximal,Average_Wage_Salaire_Moyen,Annual_Wage_Flag_Salaire_annuel
count,15.000000,27.000000,15.000000,26.000000,86.0
mean,26.404000,44.391481,76.354000,49.515385,0.0
std,2.800066,3.805085,7.630533,6.714398,0.0
min,22.290000,34.160000,59.960000,35.730000,0.0
25%,23.980000,43.230000,74.305000,46.250000,0.0
50%,26.640000,45.670000,78.430000,49.005000,0.0
75%,28.025000,46.255000,80.340000,53.030000,0.0
max,31.840000,50.950000,86.890000,69.930000,0.0


### Followed by a curated list from https://ai-jobs.net
This data source is survey based however it seems to be the main third party source for data related jobs

In [110]:
dfOG2 = pd.read_csv(r'C:\Users\Andre\OneDrive\Desktop\Data analysis\Salary_Analysis\salaries.csv')

dfOG2.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [111]:
dfOG2['employment_type'].value_counts()

employment_type
FT    13680
CT       26
PT       22
FL       12
Name: count, dtype: int64

In [112]:
#filtering for year of 2023 and fulltime employment
df2 = dfOG2[(dfOG2['work_year']>=2021) & (dfOG2['employment_type']=='FT')].copy()
df2.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,13611.000000,1.361100e+04,13611.000000,13611.000000
mean,2023.088825,1.651205e+05,150324.481596,33.234149
std,0.642918,3.556229e+05,68121.666495,46.697734
min,2021.000000,1.500000e+04,15000.000000,0.000000
25%,2023.000000,1.043000e+05,104000.000000,0.000000
50%,2023.000000,1.424000e+05,142200.000000,0.000000
75%,2023.000000,1.880000e+05,186000.000000,100.000000
max,2024.000000,3.040000e+07,800000.000000,100.000000


In [113]:
#convert prices to CAD for 2023
try:
    with open('usdcad.txt','r') as usdcadfile:
        usdcad = float(usdcadfile.read())
except:
    import MetaTrader5 as mt5
    mt5.initialize()
    mt5.symbol_select('USDCAD.p')
    prices = pd.DataFrame(mt5.copy_rates_range('USDCAD.p',mt5.TIMEFRAME_D1,pd.to_datetime('2023-01-01'),pd.to_datetime('2023-12-31')))
    usdcad =prices['close'].mean()
    with open('usdcad.txt','w') as usdcadfile:
        usdcadfile.write(str(usdcad))
df2['salary_in_cad'] = df2['salary_in_usd']*usdcad

In [114]:
#filtering for Canadian employees
df2cad = df2[df2['employee_residence'] == 'CA'].copy()

In [115]:
df2cad.drop(columns=['salary','salary_in_usd','salary_currency'],inplace=True)

In [116]:
df2cad['job_title'].value_counts()

job_title
Data Scientist                        89
Data Engineer                         56
Machine Learning Engineer             42
Data Analyst                          41
Analytics Engineer                    18
Business Intelligence Analyst         12
Machine Learning Scientist            11
Data Architect                         8
Research Scientist                     5
Data Integration Specialist            4
Machine Learning Software Engineer     4
Data Modeler                           4
ML Engineer                            4
Data Science                           4
Business Intelligence Developer        4
Data Strategist                        2
AI Programmer                          2
Machine Learning Developer             2
Director of Data Science               2
Data Developer                         2
Business Intelligence Engineer         2
Deep Learning Engineer                 2
Data Lead                              2
Research Analyst                       2
AI Eng

In [117]:
df2cad[df2cad['job_title']=='Data Analyst'].describe()

,work_year,remote_ratio,salary_in_cad
count,41.000000,41.000000,41.000000
mean,2023.000000,41.463415,144419.160009
std,0.806226,49.877900,55748.474795
min,2021.000000,0.000000,54991.278050
25%,2022.000000,0.000000,101218.392857
50%,2023.000000,0.000000,141705.750000
75%,2024.000000,100.000000,175445.214286
max,2024.000000,100.000000,279767.637857


In [118]:
dfDS = dfDS.drop(columns=['NOC_CNP','Wage_Comment_F', 'Data_Source_F', 'NOC_TITLE_FRA', 'Nom_RE'])

In [119]:
dfDS.columns

Index(['NOC_TITLE_ENG', 'prov', 'ER_Code_Code_RE', 'ER_Name',
       'Low_Wage_Salaire_Minium', 'Median_Wage_Salaire_Median',
       'High_Wage_Salaire_Maximal', 'Average_Wage_Salaire_Moyen',
       'Data_Source_E', 'Reference_Period', 'Revision_Date_Date_revision',
       'Annual_Wage_Flag_Salaire_annuel', 'Wage_Comment_E'],
      dtype='object')

In [120]:
#remove any row that has no data in any of the salary columns
dfDS.dropna(how='all',inplace=True,subset=['Low_Wage_Salaire_Minium', 'Median_Wage_Salaire_Median','High_Wage_Salaire_Maximal', 'Average_Wage_Salaire_Moyen'])

In [121]:
dfDS.reset_index(inplace=True,drop=True)
dfDS[::3]

,NOC_TITLE_ENG,prov,ER_Code_Code_RE,ER_Name,Low_Wage_Salaire_Minium,Median_Wage_Salaire_Median,High_Wage_Salaire_Maximal,Average_Wage_Salaire_Moyen,Data_Source_E,Reference_Period,Revision_Date_Date_revision,Annual_Wage_Flag_Salaire_annuel,Wage_Comment_E
0,Data scientists,NaN,ER00,Canada,25.74,45.96,78.38,52.25,2021 Census,2021,2023-11-29,0,NaN
3,Data scientists,NB,ER13,New Brunswick,NaN,37.29,NaN,46.55,2021 Census,2021,2023-11-29,0,NaN
6,Data scientists,QC,ER2435,Montérégie,29.64,45.34,59.96,45.83,2021 Census,2021,2023-11-29,0,NaN
9,Data scientists,QC,ER2460,Outaouais,NaN,44.15,NaN,45.13,2021 Census,2021,2023-11-29,0,NaN
12,Data scientists,ON,ER3530,Toronto,27.48,48.38,80.32,54.19,2021 Census,2021,2023-11-29,0,NaN
15,Data scientists,ON,ER3560,London,NaN,44.71,NaN,47.12,2021 Census,2021,2023-11-29,0,NaN
18,Data scientists,MB,ER4650,Winnipeg,NaN,41.53,NaN,36.58,2021 Census,2021,2023-11-29,0,NaN
21,Data scientists,AB,ER48,Alberta,28.57,43.14,78.61,49.85,2021 Census,2021,2023-11-29,0,NaN
24,Data scientists,BC,ER59,British Columbia,22.67,46.10,83.56,55.51,2021 Census,2021,2023-11-29,0,NaN


### Another dataset from Statscan, but more broad
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=9810041201

In [122]:
dfinc = pd.read_csv(r'C:\Users\Andre\OneDrive\Desktop\Data analysis\Salary_Analysis\income_field_of_study.csv',index_col=None)
dfinc1 = dfinc[dfinc.iloc[:,0].str.contains('Data')].copy()
dfinc1

,Occupation,Total Employment ppls,With Income,Med Income,Mean Income,Med Inc + coms,Mean Inc + coms,Average wages salaries and commissions
93,14111 Data entry clerks,"46,895","43,300","31,200","33,080","41,595","32,400","33,760"
141,21211 Data scientists,"15,420","14,450","83,000","89,500","14,080","83,000","89,600"
146,21223 Database analysts and data administrators,"26,225","25,025","72,000","74,400","24,305","72,500","75,200"


### Now we are getting the Job Bank Information from https://www.jobbank.gc.ca
Using Selenium, we will access websites like the Canadian Job Bank, or Indeed, and scrape their postings to get salary data around the country

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotVisibleException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import bs4
def jobbank_salaries(searchterms:str,file_path:str,add_type:str='a'):
    driver = webdriver.Chrome()
    start_ ='https://www.jobbank.gc.ca/jobsearch/jobsearch?searchstring='
    end_ = '&locationstring=Canada'
    searchterms = searchterms.replace(' ','+')
    searchterms_ = searchterms.rstrip()
    searchterms_ = start_+searchterms+end_
    driver.get(searchterms_)
    time.sleep(5) 
    try:
        while True:
            see_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@type="button"][@id="moreresultbutton"]')))
            if see_more_button:
                see_more_button.click()
                time.sleep(3)
            else: break
    except (NoSuchElementException, ElementNotVisibleException, TimeoutException) as e:
        print("The 'See More' button is no longer available.")
    html_source = driver.page_source
    with open(file_path, add_type, encoding='utf-8') as file:
        file.write(html_source)
    driver.quit()
    return searchterms

def indeed_salaries(searchterms:str,file_path:str,pages:int,s:bool=True):
    driver = webdriver.Chrome()
    start_ ='https://ca.indeed.com/jobs?q='
    end_ = '&l=Canada&vjk=d604f2f27a7640cf'
    searchterms = searchterms.replace(' ','+')
    searchterms_ = searchterms.rstrip()
    searchterms_ = start_+searchterms+end_
    driver.get(searchterms_)
    time.sleep(5) 
    html_source=''
    c=0
    try:
        while c<pages:
            html_source=html_source+driver.page_source
            if c==1:
                time.sleep(3)
                x_button = driver.find_element(By.XPATH,'//h3[@class="DesktopJobAlertPopup-heading"]//following::button[1]')
                x_button.click()
            see_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Next Page"]')))
            if see_more_button:
                see_more_button.click()
                time.sleep(2)
            else: break
            c+=1
    except (NoSuchElementException, ElementNotVisibleException, TimeoutException) as e:
        print("The 'See More' button is no longer available.",e)
    if s:    
        soup =bs4.BeautifulSoup(html_source,'html.parser')
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify()) 
    else:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html_source)
    driver.quit()
    return searchterms

In [17]:
filepathhtml = r'C:\Users\Andre\OneDrive\Desktop\Data analysis\Salary_Analysis\cadjobbanktest.html'
if input('run jobbank search?') == 'y':
    job = jobbank_salaries('Data Analyst',filepathhtml,add_type='w')

The 'See More' button is no longer available.


In [20]:
#for salaries with a range, take the min of the range, by finding 'to' or ' - ' and remove the tail of the str
#convert annual salaries to hourly with 40 hours a week.
def convCol(x:str):
    newx=x
    x=x.lower()
    if 'hour' in x and ('to' in x or '–' in x or '-' in x):
        if 'to' in x:
            x=x[:x.index('t')]
        elif '–' in x:
            x=x[:x.index('–')]
        else:
            x=x[:x.index('-')]
        newx = float(re.sub('[^0-9.]','',x))
    elif 'hour' in x:
        newx = float(re.sub('[^0-9.]','',x))
    elif 'month' in x and ('to' in x or '–' in x or '-'in x):
        if '-' in x:
            x=x[:x.index('-')]
        elif 'to' in x:
            x=x[:x.index('to')]
        else:
            x=x[:x.index('–')]

        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/((52/12)*40)
    elif 'month' in x:
        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/((52/12)*40)
    elif 'week' in x and ('to' in x or '–' in x or '-'in x ):
        if '-' in x:
            x=x[:x.index('-')]
        elif 't' in x:
            x=x[:x.index('t')]
        else:
            x=x[:x.index('–')]
        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/40
    elif 'week' in x:
        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/40
    elif ('annual' in x or 'year'in x)  and ('to' in x or '–' in x or '-' in x ):
        if 'to' in x :
            x=x[:x.index('t')]
        elif '-' in x:
            x=x[:x.index('-')]
        else:
            x=x[:x.index('–')]
        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/(40*52)    
    elif 'annual' in x or 'year' in x:
        newx = float(re.sub('[^0-9.]','',x))
        newx=newx/(40*52)
    else:
        newx = float(re.sub('[^0-9.]','',x))
    return newx

In [51]:
def jobbank_cleaning(file:str):
    with open(file, 'r',encoding='utf-8') as data:
        rawhtml = data.read()
    articles = re.findall('<article.*?</article>',rawhtml,re.DOTALL)
    print(f"Amount of items found: {len(articles)}")
    jobs = {'Title':[],'Date':[],'Salary':[],'Location':[],'Company':[]}
    for i in articles:
        try:
            salary = (re.search('salary.+?Salary:\n(.+?)<',i,re.DOTALL)).group(1).lstrip().rstrip()
        except Exception as e:
            continue
        title = (re.search('noctitle">(.+?)\n',i)).group(1).lstrip().rstrip()
        location=(re.search('Location</span>(.+?)<',i,re.DOTALL)).group(1).lstrip().rstrip()
        date = (re.search('"date">(.*?)\n',i)).group(1).lstrip().rstrip()
        company = (re.search('business">(.+?)<',i)).group(1).lstrip().rstrip()
        jobs['Title'].append(title)
        jobs['Company'].append(company)
        jobs['Location'].append(location)
        jobs['Date'].append(date)
        jobs['Salary'].append(salary)
    dfjobbankOG = pd.DataFrame(jobs)
    dfjobbank =dfjobbankOG.copy()
    dfjobbank['Salary']=dfjobbank['Salary'].apply(convCol)
    dfjobbank['City'] = dfjobbank['Location'].str[:-4]
    dfjobbank['Province'] = dfjobbank['Location'].str[-4:]
    dfjobbank.drop(columns='Location',inplace=True)
    dfjobbank = dfjobbank[['Title','Salary','City','Province','Company','Date']] 
    print(f"Amount with Salaries: {len(dfjobbank)}")
    return (dfjobbank,dfjobbankOG)

In [52]:
dfjobbank,dfjobbankOG=jobbank_cleaning(filepathhtml)

Amount of items found: 309
Amount with Salaries: 197


In [53]:
dfjobbank['Province'].value_counts()

Province
(ON)    89
(AB)    46
(BC)    40
(QC)    13
(SK)     3
(MB)     2
(NS)     2
(NL)     1
(NB)     1
Name: count, dtype: int64

In [54]:
dfjobbank.groupby('Province')['Salary'].mean().sort_values(ascending=False)

Province
(QC)    3774.199985
(AB)      42.958261
(ON)      42.353272
(NL)      42.300000
(SK)      40.146667
(BC)      39.431058
(NS)      36.826923
(NB)      35.500000
(MB)      30.665000
Name: Salary, dtype: float64

Obviously here there seems to be an issue with salaries in QC, so let's investigate what might be the problem with 

`dfjobbank['Salaries]>150`


In [61]:
dfjobbank[dfjobbank['Salary']>150].index

Index([9], dtype='int64')

In [62]:
dfjobbankOG.loc[dfjobbank[dfjobbank['Salary']>150].index]

,Title,Date,Salary,Location,Company
9,DBA (database analyst),"March 06, 2024","$48,689.00 to $83,462.00 hourly",Gatineau (QC),Cégep Héritage College


So here we can see that: 
> Salary      $48,689.00 to $83,462.00 hourly

We can assume this data was incorrectly labeled as the hourly wages in the source data, and therefor it would need to correctly be labeled as anually

In [63]:
dfjobbank.at[9,'Salary'] = 48689/(52*40)
dfjobbank.groupby('Province')['Salary'].mean().sort_values(ascending=False)

Province
(AB)    42.958261
(ON)    42.353272
(NL)    42.300000
(SK)    40.146667
(BC)    39.431058
(NS)    36.826923
(NB)    35.500000
(QC)    30.692922
(MB)    30.665000
Name: Salary, dtype: float64

### Indeed job search 

In [57]:
if input('run indeed search?') == 'y':
    indeed_salaries('Data Analysis','test.html',25,True)

In [58]:
def indeed_cleaning(file:str):
    with open(file, 'r',encoding='utf-8') as file:
        rawhtml = file.read()
    tbodies = re.findall('<tbody.*?</tbody>',rawhtml,re.DOTALL)
    print(f"Amount of items found: {len(tbodies)}")
    indeedjobs={'Salary':[],'Name':[],'Company':[],'City':[],'Province':[]}
    for i in tbodies:
        try:
            salary = (re.search('(\$.+?)\n',i,re.DOTALL)).group(1)
            indeedjobs['Salary'].append(salary)
        except Exception as e:
            indeedjobs['Salary'].append(float('NaN'))
        try:
            name = (re.search('title="(.+?)"',i)).group(1)
            indeedjobs['Name'].append(name)
        except:
            indeedjobs['Name'].append(None)
        try:
            company = (re.search('company-name.+?\n(.+?)\n',i,re.DOTALL)).group(1)
            indeedjobs['Company'].append(company)
        except:
            indeedjobs['Company'].append(None)
        try:
            location = (re.search('text-location.*?\n(.*?)\n',i,re.DOTALL)).group(1)
            city,province = location.split(',')
            indeedjobs['City'].append(city)
            indeedjobs['Province'].append(province.lstrip())
        except:
            indeedjobs['City'].append(None)
            indeedjobs['Province'].append(None)
        

    dfindeedOG = pd.DataFrame(indeedjobs)
    dfindeed = dfindeedOG.copy()
    dfindeed.dropna(how='any',inplace=True)
    dfindeed.reset_index(inplace=True,drop=True)
    dfindeed['Salary'] = dfindeed['Salary'].apply(convCol)
    print(f"Amount with Salaries {len(dfindeed)}")
    return (dfindeed,dfindeedOG)

In [59]:
dfindeed,dfindeedOG = indeed_cleaning('test.html')
dfindeed['Province'].value_counts()

Amount of items found: 750
Amount with Salaries 122


Province
ON    39
BC    36
AB    22
QC    14
NS     7
MB     4
Name: count, dtype: int64

In [60]:
dfindeed.groupby('Province')['Salary'].mean().sort_values(ascending=False)

Province
QC    65.465664
BC    41.136995
MB    40.384615
AB    37.699650
ON    36.140794
NS    36.057692
Name: Salary, dtype: float64